# The Validation Experiment Design

## 1.3 Implementing the confusion detection experiment

In [1]:
def run_confusion_validation():
    """
    the experiment that either proves
    or disapproves confusion detection:

    - phase 1: train on addition -> low confusion
    within architecture capability
    - phase 2a: introduce multiplication -> high confusion
    proving necessity for new architecture
    - phase 2b: introduce subtraction -> low confusion
    proving same architecture works

    if we can't distinguish 2a from 2b, 
    the whole project sucks
    """

    model = InstrumentedTransformer(ModelConfig())
    detector = ConfusionDetector()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    results = {
        'phase1_addition': [],
        'phase2a_multiplication': [],
        'phase2b_subtraction': []
    }

    # phase 1: addition training to establish baseline
    print("Phase 1: Training on addition...")
    for step in range(500):
        batch = generate_arithmetic_batch(operation='add', batch_size=32)

        # forward w/ instrumentation
        logits, metadata = model(batch['input'], return_metadata=True)
        loss = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)),
                                     batch['target'].view(-1))

        # compute confusion metrics
        metrics = detector.compute_all_metrics(model, batch, metadata)
        results['phase1_addition'].append(metrics)

        # train!
        optimizer.zero_grad()
        loss_backward()
        optimizer.step()

        if step % 50 == 0:
            print(f"Step {step}, Loss: {loss.item():.4f}, "
                  f"Entropy: {metrics['attention_entropy']:.4f}")

    print("Addition learned. Confusion metrics baseline established.\n")

    # phase 2a: introduce multiplication
    # which should spike confusion
    print("Phase 2a: introducing multiplication...")
    for step in range